In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from custom_models import cnn
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time
import os
from PIL import Image
from torchvision.transforms import v2
from tempfile import TemporaryDirectory
import shutil
import random

cudnn.benchmark = True
plt.ion()   # interactive mode

#### Data Transformations

In [3]:
# data transformations to loop through
minimal_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

basic_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.RandomHorizontalFlip(p=0.5),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

auto_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.AutoAugment(policy=v2.AutoAugmentPolicy.IMAGENET),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

#### Create Training Set with Real/Synthetic Images

In [4]:
def makeSyntheticTrain(train_directory, synthetic_train_directory, train_percentage, synthetic_percentage):

    # Remove any existing images in directory
    try:
        shutil.rmtree(synthetic_train_directory)
    except:
        print("directory does not exist")

    # Loop through subfolders, generate synthetic images
    subfolders = [f for f in os.listdir(train_directory)]

    for s in subfolders:
        # for each subfolder in the train directory, make the same in the synthetic train directory
        os.makedirs(f"{synthetic_train_directory}/{s}", exist_ok=True)
        
        # get a random sample from each subfolder
        subfolder_path = f"{train_directory}/{s}"
        files = os.listdir(subfolder_path)
        sample_files = random.sample(files, round(len(files)*train_percentage))
        
        # create synthetic sample based on sampled original images
        synthetic_subfolder_path = subfolder_path.replace('train','synthetic')
        synthetic_files = [f for f in os.listdir(synthetic_subfolder_path) if int(f.replace('.png','').split('_')[1]) in [int(f.replace('.png','').split('_')[1]) for f in sample_files]]
        synthetic_sample_files = random.sample(synthetic_files, round(len(files)*synthetic_percentage))
        
        # Move sample files to synthetic directory
        for f in sample_files:
            
            image_path = f"{subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

        # Move synthetic sample files to synthetic directory
        for f in synthetic_sample_files:

            image_path = f"{synthetic_subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

#### Read Data

In [5]:
def get_data(data_dir, data_sets, data_transforms):
    
    image_datasets = {
        x: datasets.ImageFolder(
            os.path.join(data_dir, x),
            data_transforms[x]
        )
        for x in data_sets
    }

    dataloaders = {
        x: DataLoader(
            image_datasets[x], 
            batch_size=4,
            shuffle=True, 
            num_workers=4
        )
        for x in data_sets
    }

    dataset_sizes = {
        x: len(image_datasets[x]) 
        for x in data_sets
    }

    class_names = image_datasets['synthetic_train'].classes

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    return image_datasets, dataloaders, dataset_sizes, class_names, device

#### Train Model

In [6]:
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs=10):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        
        best_test_loss = float('inf')
        patience = 2  # Number of epochs to wait for improvement before stopping
        test_losses = []
        train_losses = []
        test_acc = []
        train_acc = []
        patience_counter = 0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['synthetic_train', 'test']:
                if phase == 'synthetic_train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'synthetic_train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'synthetic_train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double().item() / dataset_sizes[phase]

                if phase =='synthetic_train':
                    train_losses.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    test_losses.append(epoch_loss)
                    test_acc.append(epoch_acc)
                    
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'test' and epoch_loss <= best_test_loss:
                    best_test_loss = epoch_loss
                    patience_counter = 0  # Reset counter
                    torch.save(model.state_dict(), best_model_params_path)
                elif phase == 'test' and epoch_loss > best_test_loss:
                    patience_counter += 1
                
                # Early stopping check
                #if patience_counter >= patience:
                #    print("Stopping early due to no improvement in validation loss.")
                #    break

        # store results in dataframe
        dat = {
            "epoch": range(len(test_losses)),
            "test_losses": test_losses,
            "train_losses": train_losses,
            "test_accuracies": test_acc,
            "train_accuracies": train_acc
        }

        result = pd.DataFrame(data=dat)
        print()
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best test Loss: {best_test_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return result, model

#### Loop through different training scenarios

In [8]:
num_epochs = 10
num_sims = 10
train_percs = [0.8]#[1.0, 0.8, 0.8, 0.6, 0.6, 0.4, 0.4]
synth_percs = [0.0]#[0.0, 0.2, 0.0, 0.4, 0.0, 0.6, 0.0]
transforms = {
    'minimal': minimal_transforms,
    'basic': basic_transforms,
    'auto': auto_transforms
}

for train_percentage, synthetic_percentage in zip(train_percs, synth_percs):
    for active_transform in transforms.keys():
        df_all_results = pd.DataFrame()
        for n in range(num_sims):
    
            # make the synthetic training dataset
            makeSyntheticTrain(
                train_directory='../data/alzheimer_mri/train',
                synthetic_train_directory='../data/alzheimer_mri/synthetic_train', 
                train_percentage=train_percentage, 
                synthetic_percentage=synthetic_percentage
            )

            # get and load datasets
            data_dir = '../data/alzheimer_mri'
            data_sets = ['synthetic_train','test']
            image_datasets, dataloaders, dataset_sizes, class_names, device = get_data(
                data_dir=data_dir, 
                data_sets=data_sets, 
                data_transforms=transforms.get(active_transform)
            )

            # instantiate model
            model = cnn.CNN(in_channels=1, num_classes=4)

            # Set the size of each output sample to nn.Linear(num_ftrs, len(class_names))
            #num_ftrs = 4 #model.fc.in_features
            #model.fc = nn.Linear(num_ftrs, len(class_names))
            model = model.to(device)

            # train the model
            df_results,_ = model = train_model(
                model=model, 
                criterion = nn.CrossEntropyLoss(),
                optimizer = optim.Adam(model.parameters(), lr=0.001), 
                dataloaders=dataloaders,
                dataset_sizes=dataset_sizes,
                num_epochs=num_epochs
            )

            df_results['train_percentage'] = train_percentage
            df_results['synth_percentage'] = synthetic_percentage
            df_results['transform'] = active_transform
            df_results['sim_num'] = n
            df_results['category'] = df_results.apply(lambda row: row['transform']+'_'+str(row['train_percentage'])+'_'+str(row['synth_percentage']), axis=1)
            df_all_results = pd.concat([df_all_results, df_results],ignore_index=True)
        df_all_results.to_csv('../results/results_cnn_'+active_transform+'_'+str(train_percentage)+'_'+str(synthetic_percentage)+'.csv')

/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 0.9197 Acc: 0.5508
test Loss: 0.8188 Acc: 0.6031
Epoch 1/9
----------
synthetic_train Loss: 0.5635 Acc: 0.7515
test Loss: 0.4625 Acc: 0.8336
Epoch 2/9
----------
synthetic_train Loss: 0.1963 Acc: 0.9255
test Loss: 0.3248 Acc: 0.8906
Epoch 3/9
----------
synthetic_train Loss: 0.0959 Acc: 0.9670
test Loss: 0.2861 Acc: 0.9133
Epoch 4/9
----------
synthetic_train Loss: 0.0319 Acc: 0.9902
test Loss: 0.3367 Acc: 0.9156
Epoch 5/9
----------
synthetic_train Loss: 0.0417 Acc: 0.9844
test Loss: 0.3753 Acc: 0.9086
Epoch 6/9
----------
synthetic_train Loss: 0.0394 Acc: 0.9863
test Loss: 0.2986 Acc: 0.9281
Epoch 7/9
----------
synthetic_train Loss: 0.0148 Acc: 0.9966
test Loss: 0.3795 Acc: 0.9328
Epoch 8/9
----------
synthetic_train Loss: 0.0465 Acc: 0.9851
test Loss: 0.3093 Acc: 0.9336
Epoch 9/9
----------
synthetic_train Loss: 0.0267 Acc: 0.9910
test Loss: 0.3138 Acc: 0.9297

Training complete in 2m 47s
Best test Loss: 0.286139


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 0.9278 Acc: 0.5552
test Loss: 0.9897 Acc: 0.4398
Epoch 1/9
----------
synthetic_train Loss: 0.6318 Acc: 0.7205
test Loss: 0.5197 Acc: 0.7773
Epoch 2/9
----------
synthetic_train Loss: 0.2732 Acc: 0.8894
test Loss: 0.2941 Acc: 0.8867
Epoch 3/9
----------
synthetic_train Loss: 0.1044 Acc: 0.9590
test Loss: 0.3090 Acc: 0.8969
Epoch 4/9
----------
synthetic_train Loss: 0.0570 Acc: 0.9783
test Loss: 0.3126 Acc: 0.9227
Epoch 5/9
----------
synthetic_train Loss: 0.0510 Acc: 0.9817
test Loss: 0.3535 Acc: 0.9047
Epoch 6/9
----------
synthetic_train Loss: 0.0210 Acc: 0.9927
test Loss: 0.3301 Acc: 0.9336
Epoch 7/9
----------
synthetic_train Loss: 0.0429 Acc: 0.9866
test Loss: 0.6393 Acc: 0.8375
Epoch 8/9
----------
synthetic_train Loss: 0.0211 Acc: 0.9946
test Loss: 0.2873 Acc: 0.9383
Epoch 9/9
----------
synthetic_train Loss: 0.0207 Acc: 0.9939
test Loss: 0.3072 Acc: 0.9187

Training complete in 2m 53s
Best test Loss: 0.287338


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Epoch 0/9
----------
synthetic_train Loss: 1.0693 Acc: 0.4897
test Loss: 1.0170 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0033 Acc: 0.5146
test Loss: 0.8990 Acc: 0.5773
Epoch 2/9
----------
synthetic_train Loss: 0.9567 Acc: 0.5349
test Loss: 0.9032 Acc: 0.5711
Epoch 3/9
----------
synthetic_train Loss: 0.9310 Acc: 0.5608
test Loss: 0.8049 Acc: 0.6031
Epoch 4/9
----------
synthetic_train Loss: 0.8527 Acc: 0.6086
test Loss: 0.6977 Acc: 0.7086
Epoch 5/9
----------
synthetic_train Loss: 0.7907 Acc: 0.6375
test Loss: 0.5721 Acc: 0.7734
Epoch 6/9
----------
synthetic_train Loss: 0.7379 Acc: 0.6665
test Loss: 0.6231 Acc: 0.7781
Epoch 7/9
----------
synthetic_train Loss: 0.6579 Acc: 0.7109
test Loss: 0.3952 Acc: 0.8375
Epoch 8/9
----------
synthetic_train Loss: 0.5960 Acc: 0.7478
test Loss: 0.3146 Acc: 0.8844
Epoch 9/9
----------
synthetic_train Loss: 0.5293 Acc: 0.7676
test Loss: 0.3733 Acc: 0.8641

Training complete in 3m 20s
Best test Loss: 0.314567


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0602 Acc: 0.4929
test Loss: 1.0181 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0279 Acc: 0.4998
test Loss: 0.9288 Acc: 0.5312
Epoch 2/9
----------
synthetic_train Loss: 0.9746 Acc: 0.5334
test Loss: 0.8950 Acc: 0.5578
Epoch 3/9
----------
synthetic_train Loss: 0.9306 Acc: 0.5369
test Loss: 0.8735 Acc: 0.5773
Epoch 4/9
----------
synthetic_train Loss: 0.8875 Acc: 0.5779
test Loss: 0.7772 Acc: 0.6164
Epoch 5/9
----------
synthetic_train Loss: 0.8275 Acc: 0.6060
test Loss: 0.6972 Acc: 0.6867
Epoch 6/9
----------
synthetic_train Loss: 0.7580 Acc: 0.6404
test Loss: 0.5626 Acc: 0.7453
Epoch 7/9
----------
synthetic_train Loss: 0.7040 Acc: 0.6829
test Loss: 0.3848 Acc: 0.8508
Epoch 8/9
----------
synthetic_train Loss: 0.6298 Acc: 0.7173
test Loss: 0.4816 Acc: 0.8227
Epoch 9/9
----------
synthetic_train Loss: 0.5967 Acc: 0.7307
test Loss: 0.5154 Acc: 0.8180

Training complete in 3m 4s
Best test Loss: 0.384770


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0586 Acc: 0.4854
test Loss: 1.0064 Acc: 0.4953
Epoch 1/9
----------
synthetic_train Loss: 1.0063 Acc: 0.5071
test Loss: 0.8861 Acc: 0.5844
Epoch 2/9
----------
synthetic_train Loss: 0.9486 Acc: 0.5449
test Loss: 0.8439 Acc: 0.5859
Epoch 3/9
----------
synthetic_train Loss: 0.8899 Acc: 0.5706
test Loss: 0.7655 Acc: 0.6578
Epoch 4/9
----------
synthetic_train Loss: 0.8334 Acc: 0.6133
test Loss: 0.7076 Acc: 0.7047
Epoch 5/9
----------
synthetic_train Loss: 0.7580 Acc: 0.6477
test Loss: 0.5997 Acc: 0.7523
Epoch 6/9
----------
synthetic_train Loss: 0.6946 Acc: 0.6819
test Loss: 0.5127 Acc: 0.7977
Epoch 7/9
----------
synthetic_train Loss: 0.6682 Acc: 0.6978
test Loss: 0.4350 Acc: 0.8344
Epoch 8/9
----------
synthetic_train Loss: 0.6152 Acc: 0.7222
test Loss: 0.3691 Acc: 0.8625
Epoch 9/9
----------
synthetic_train Loss: 0.5673 Acc: 0.7480
test Loss: 0.3614 Acc: 0.8828

Training complete in 3m 3s
Best test Loss: 0.361422
